# Proyecto 1: Análisis de Datos de Defunciones INE (2015-2024)

**Institución:** Universidad del Valle de Guatemala  
**Curso:** Minería de Datos  
**Fecha:** 5 de febrero de 2026  
**Autores:** Diego López, Erick Guerra, Diego Rosales

## Descripción
Análisis estadístico de datos de defunciones reportados por el Instituto Nacional de Estadística (INE) durante un período de 10 años (2015-2024).

## Objetivos
- Explorar y visualizar tendencias de defunciones
- Análisis estadístico descriptivo
- Identificar patrones y variaciones temporales

## 1. Importar Librerías Necesarias

In [15]:
import json
import pandas as pd
import numpy as np
from pathlib import Path
from typing import Dict, List, Tuple
import warnings
warnings.filterwarnings('ignore')

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 1.2. Verificar y Generar Datos Necesarios

In [ ]:
import subprocess
import sys

# Ruta del archivo de datos
datos_json_path = Path("data/json/datos_completos.json")

if not datos_json_path.exists():
    print("⚠ El archivo datos_completos.json no existe.")
    print("Ejecutando scripts en orden...\n")
    
    # 1. Generar mapeo
    print("1️ Ejecutando generar_mapeo.py...")
    resultado = subprocess.run([sys.executable, "src/generar_mapeo.py"], capture_output=True, text=True)
    if resultado.returncode == 0:
        print("✓ Mapeo generado correctamente\n")
    else:
        print(f"✗ Error: {resultado.stderr}\n")
    
    # 2. Extraer estructura
    print("2️ Ejecutando extraer_estructura.py...")
    resultado = subprocess.run([sys.executable, "src/extraer_estructura.py"], capture_output=True, text=True)
    if resultado.returncode == 0:
        print("✓ Estructura extraída correctamente\n")
    else:
        print(f"✗ Error: {resultado.stderr}\n")
    
    # 3. Extraer datos
    print("3️ Ejecutando extraer_datos.py...")
    resultado = subprocess.run([sys.executable, "src/extraer_datos.py"], capture_output=True, text=True)
    if resultado.returncode == 0:
        print("✓ Datos extraídos correctamente\n")
    else:
        print(f"✗ Error: {resultado.stderr}\n")
    
    print("✓ Todos los scripts han sido ejecutados correctamente")
else:
    print(f"✓ El archivo {datos_json_path} ya existe")

✓ El archivo data\json\datos_completos.json ya existe


## 2. Configurar Rutas y Cargar Archivo de Datos

In [17]:
# Rutas de archivos
datos_json = 'data/json/datos_completos.json'
estructura_json = 'data/json/estructura_completa.json'
mapeo_json = 'data/json/mapeo_hojas.json'

# Cargar datos
print("Cargando datos...")
with open(datos_json, 'r', encoding='utf-8') as f:
    datos = json.load(f)

with open(estructura_json, 'r', encoding='utf-8') as f:
    estructura = json.load(f)

with open(mapeo_json, 'r', encoding='utf-8') as f:
    mapeo = json.load(f)

print(f"✓ Datos cargados correctamente")
print(f"  - Temáticas: {len(datos)}")
print(f"  - Años disponibles: {set().union(*[set(datos[t].keys()) for t in datos])}")

Cargando datos...
✓ Datos cargados correctamente
  - Temáticas: 16
  - Años disponibles: {'2017', '2022', '2019', '2016', '2018', '2021', '2024', '2015', '2023', '2020'}


## 3. Crear DataFrames por Temática

In [18]:
def crear_dataframe_tematica(tematica_nombre: str, datos: dict, años: List[str] = None) -> Dict[str, pd.DataFrame]:
    """
    Convierte los datos de una temática en DataFrames por año.
    
    Args:
        tematica_nombre: Nombre de la temática
        datos: Diccionario con datos de la temática
        años: Lista de años a incluir (si None, incluye todos)
    
    Returns:
        Dict con DataFrames por año: {año: DataFrame}
    """
    dfs = {}
    
    if años is None:
        años = list(datos[tematica_nombre].keys())
    
    for año in años:
        if año not in datos[tematica_nombre]:
            continue
        
        data_año = datos[tematica_nombre][año]
        
        # Crear DataFrame a partir de los datos
        if data_año['datos']:
            df = pd.DataFrame(data_año['datos'])
            df['año'] = int(año)  # Agregar columna de año
            dfs[año] = df
    
    return dfs

# Crear diccionario para almacenar todos los DataFrames
dataframes_por_tema = {}

print("Creando DataFrames por temática...")
for idx, tematica in enumerate(datos.keys(), 1):
    dataframes_por_tema[tematica] = crear_dataframe_tematica(tematica, datos)
    print(f"  [{idx:2d}] {tematica[:60]}...")

print(f"\n✓ {len(dataframes_por_tema)} temáticas procesadas")

Creando DataFrames por temática...
  [ 1] Defunciones por año de ocurrencia, según departamento de res...
  [ 2] Defunciones por departamento de ocurrencia, según departamen...
  [ 3] Defunciones por sexo, según departamento de residencia del d...
  [ 4] Defunciones por sexo, según departamento de residencia del d...
  [ 5] Defunciones por sexo, según departamento de residencia del d...
  [ 6] Defunciones por sexo, según edad y causas de muerte...
  [ 7] Defunciones por sexo, según departamento de residencia del d...
  [ 8] Defunciones por tipo de certificación, según departamento y ...
  [ 9] Defunciones por tipo de asistencia recibida, según departame...
  [10] Defunciones por lugar de ocurrencia, según departamento y mu...
  [11] Defunciones infantiles, neonatales y postneonatales por sexo...
  [12] Defunciones neonatales por sexo, según edad y causas de muer...
  [13] Defunciones postneonatales por sexo, según edad y causas de ...
  [14] Defunciones por mes de ocurrencia,  según dí

## 4. Explorar y Acceder a los Datos

In [19]:
def listar_tematicas():
    """Lista todas las temáticas disponibles"""
    print("Temáticas disponibles:")
    for idx, tematica in enumerate(dataframes_por_tema.keys(), 1):
        años_disponibles = list(dataframes_por_tema[tematica].keys())
        print(f"  {idx:2d}. {tematica[:70]}")
        print(f"      Años: {', '.join(años_disponibles)}")

def obtener_dataframe(tematica: str, año: str) -> pd.DataFrame:
    """Obtiene un DataFrame específico de una temática y año"""
    if tematica not in dataframes_por_tema:
        raise ValueError(f"Temática '{tematica}' no encontrada")
    if año not in dataframes_por_tema[tematica]:
        raise ValueError(f"Año '{año}' no disponible para '{tematica}'")
    return dataframes_por_tema[tematica][año]

def combinar_años(tematica: str, años: List[str] = None) -> pd.DataFrame:
    """Combina DataFrames de múltiples años en una temática"""
    if tematica not in dataframes_por_tema:
        raise ValueError(f"Temática '{tematica}' no encontrada")
    
    if años is None:
        años = list(dataframes_por_tema[tematica].keys())
    
    dfs = [dataframes_por_tema[tematica][año] for año in años 
           if año in dataframes_por_tema[tematica]]
    
    if not dfs:
        raise ValueError("No hay datos para combinar")
    
    return pd.concat(dfs, ignore_index=True)

# Ejemplo: Listar todas las temáticas
listar_tematicas()

Temáticas disponibles:
   1. Defunciones por año de ocurrencia, según departamento de residencia de
      Años: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   2. Defunciones por departamento de ocurrencia, según departamento de resi
      Años: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   3. Defunciones por sexo, según departamento de residencia del difunto(a) 
      Años: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   4. Defunciones por sexo, según departamento de residencia del difunto(a) 
      Años: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   5. Defunciones por sexo, según departamento de residencia del difunto(a),
      Años: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   6. Defunciones por sexo, según edad y causas de muerte
      Años: 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023, 2024
   7. Defunciones por sexo, según departamento de residencia del difunto(a) 
      Años: 2015, 2016, 2017, 2

## 5. Ejemplos de Uso

### Ejemplo 1: Obtener un DataFrame específico

In [20]:
# Obtener la primera temática disponible
primera_tematica = list(dataframes_por_tema.keys())[0]
print(f"Temática: {primera_tematica}\n")

# Obtener DataFrame de 2015
df_2015 = obtener_dataframe(primera_tematica, '2015')
print(f"DataFrame 2015 - Shape: {df_2015.shape}")
print(f"Columnas: {list(df_2015.columns)}\n")
print(df_2015.head())

Temática: Defunciones por año de ocurrencia, según departamento de residencia del difunto(a)

DataFrame 2015 - Shape: (27, 7)
Columnas: ['Departamento de residencia', 'Año de ocurrencia_2019', 'Año de ocurrencia_2020', 'Año de ocurrencia_2021', 'Año de ocurrencia_2022', 'Año de ocurrencia_2023', 'año']

  Departamento de residencia  Año de ocurrencia_2019  Año de ocurrencia_2020  \
0                 Total País                 85600.0                 96001.0   
1                  Guatemala                 18820.0                 23719.0   
2                El Progreso                  1126.0                  1214.0   
3               Sacatepéquez                  1783.0                  2061.0   
4              Chimaltenango                  2984.0                  3113.0   

   Año de ocurrencia_2021  Año de ocurrencia_2022  Año de ocurrencia_2023  \
0                118465.0                 95386.0                 95948.0   
1                 27445.0                 21281.0           

### Ejemplo 2: Combinar años para análisis de serie temporal

In [21]:
# Combinar todos los años para análisis de serie temporal
df_completo = combinar_años(primera_tematica)
print(f"DataFrame combinado (2015-2024) - Shape: {df_completo.shape}")
print(f"\nAños incluidos: {sorted(df_completo['año'].unique())}")
print(f"\nData types:\n{df_completo.dtypes}")

DataFrame combinado (2015-2024) - Shape: (270, 8)

Años incluidos: [np.int64(2015), np.int64(2016), np.int64(2017), np.int64(2018), np.int64(2019), np.int64(2020), np.int64(2021), np.int64(2022), np.int64(2023), np.int64(2024)]

Data types:
Departamento de residencia     object
Año de ocurrencia_2019        float64
Año de ocurrencia_2020        float64
Año de ocurrencia_2021        float64
Año de ocurrencia_2022        float64
Año de ocurrencia_2023        float64
año                             int64
Año de ocurrencia_2024        float64
dtype: object


## 6. Resumen Estadístico de los Datos

In [22]:
print("=" * 80)
print("RESUMEN ESTADÍSTICO")
print("=" * 80)

total_filas_todos = 0
print("\nTemáticas procesadas:")
for idx, (tematica, dfs_años) in enumerate(dataframes_por_tema.items(), 1):
    filas_tematica = sum(len(df) for df in dfs_años.values())
    total_filas_todos += filas_tematica
    print(f"  {idx:2d}. {tematica[:65]} - {filas_tematica:,} filas")

print(f"\n{'='*80}")
print(f"Total de filas en toda la base de datos: {total_filas_todos:,}")
print(f"Total de temáticas: {len(dataframes_por_tema)}")
print(f"Total de años cubiertos: 10 (2015-2024)")
print(f"{'='*80}")

RESUMEN ESTADÍSTICO

Temáticas procesadas:
   1. Defunciones por año de ocurrencia, según departamento de residenc - 270 filas
   2. Defunciones por departamento de ocurrencia, según departamento de - 270 filas
   3. Defunciones por sexo, según departamento de residencia del difunt - 25,510 filas
   4. Defunciones por sexo, según departamento de residencia del difunt - 5,760 filas
   5. Defunciones por sexo, según departamento de residencia del difunt - 8,760 filas
   6. Defunciones por sexo, según edad y causas de muerte - 4,870 filas
   7. Defunciones por sexo, según departamento de residencia del difunt - 4,510 filas
   8. Defunciones por tipo de certificación, según departamento y munic - 3,670 filas
   9. Defunciones por tipo de asistencia recibida, según departamento y - 3,670 filas
  10. Defunciones por lugar de ocurrencia, según departamento y municip - 3,670 filas
  11. Defunciones infantiles, neonatales y postneonatales por sexo, seg - 2,510 filas
  12. Defunciones neonatales